## Technical requirements

```bash
conda create -n gee python
conda activate gee
conda install -c conda-forge mamba
mamba install -c conda-forge pygis
```


In [ ]:
#%pip install pygis

In [1]:
import ee
import geemap
import leafmap
#leafmap.update_package()

In [2]:
geemap.ee_initialize()

## Load data

In [40]:
WorldPop = ee.ImageCollection("WorldPop/GP/100m/pop")
pop_sahel = ee.Image("projects/ee-aboubacarhema94/assets/ACLED/Sahel_pop_2020_UNadj_constrained")
sahel_adm2 = ee.FeatureCollection("projects/ee-aboubacarhema94/assets/ACLED/G5_Sahel_adm2")
events_diffusion = ee.FeatureCollection("projects/ee-aboubacarhema94/assets/ACLED/G5_Sahel_Diffusion")
geometry = ee.Geometry.Polygon(
        [[[-19.7019400534855, 29.93665627910383],
          [-19.7019400534855, 5.813630636937255],
          [26.967981821514496, 5.813630636937255],
          [26.967981821514496, 29.93665627910383]]])

pop_sahel = ee.Image(pop_sahel)

In [13]:
viz_pop = {
  'min': 0.0,
  'max': 250.0,
  'palette': ['24126c', '1fff4f', 'd4ff50'],
}

In [24]:
#Extract the projection before doing any computation
projection = pop_sahel.projection()
#print(ee.Number(projection.nominalScale()))

In [41]:
Map = geemap.Map()
# Creat an interactive map
#Map = Map.addLayer(WorldPop, viz_pop, 'Population Sahel', 0);
Map.addLayer(pop_sahel, viz_pop, 'Population Sahel', True, 1)
Map.addLayer(sahel_adm2, {'color': 'purple'}, "sahel adm2")
Map.centerObject(sahel_adm2)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [81]:
#print(pop_sahel.aggregate_sum('b1'))
# Compute sums of the specified properties.
'''
properties = ['b1', 'b1']
sums = pop_sahel.filter(ee.Filter.notNull(properties)).reduceColumns(
    **{'reducer': ee.Reducer.sum().repeat(2), 'selectors': properties}
)
sums
'''

"\nproperties = ['b1', 'b1']\nsums = pop_sahel.filter(ee.Filter.notNull(properties)).reduceColumns(\n    **{'reducer': ee.Reducer.sum().repeat(2), 'selectors': properties}\n)\nsums\n"

## Start conflict diffusion indicator computation

In [44]:

# year of event
year = 2018

#
pop_seuil = 50

#
nb_events = 3

1. Create a spatial grid of 10km-by-10km

2. Remove grid cells with less than 100 habitants using population data from WorldPop (pop_country)

3. Identify grid cells with at least 10 conflict events per year using events_diffusion layer

3.1 Number of Conflict event

In [56]:
import geemap.chart as chart

features = events_diffusion.select('event_type')
df = geemap.ee_to_df(features, sort_columns=True)
df

,event_type
0,Battles
1,Explosions/Remote violence
2,Battles
3,Battles
4,Battles
...,...
15232,Explosions/Remote violence
15233,Violence against civilians
15234,Violence against civilians
15235,Explosions/Remote violence


In [55]:
events_diffusion_country_year = events_diffusion.filter(ee.Filter.eq('year', year))

#calculating the number of non-null values
#print('Number of Conflict event',
#      events_diffusion_country_year.aggregate_count('event_type'));
      


3.2 Vizualise Conflict event layer

In [49]:
# Use style() to visualize the points
#events_diffusion_country_year
eventsStyled = {
    'color': 'red',
    'pointSize': 1,
    'pointShape': 'triangle',
    'width': 3,
}

Map.addLayer(events_diffusion_country_year.style(**eventsStyled), {}, "conflict events")
Map

Map(bottom=3935.0, center=[19.84939395842279, -0.5053710937500001], controls=(WidgetControl(options=['position…